# 빅데이터 분석기사 실기 작업 1유형 연습문제

In [1]:
# 라이브러리 호출
import pandas as pd
import numpy as np

In [2]:
# 31. 서울시 따릉이 이용정보 데이터
# 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜 출력
# 데이터 불러오기
csvFile = 'https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv'
df = pd.read_csv(csvFile)
df.head()

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678950 entries, 0 to 678949
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   대여일자    678950 non-null  object 
 1   대여시간    678950 non-null  int64  
 2   대여소번호   678950 non-null  int64  
 3   대여구분코드  678950 non-null  object 
 4   성별      678950 non-null  object 
 5   연령대코드   678950 non-null  object 
 6   이용건수    678950 non-null  int64  
 7   운동량     678950 non-null  object 
 8   탄소량     678950 non-null  object 
 9   이동거리    678950 non-null  float64
 10  사용시간    678950 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 57.0+ MB


In [4]:
# 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜 출력
result = df['대여일자'].value_counts().sort_index().to_frame()
ans = result[result['대여일자'] == result['대여일자'].max()].index[0]
display(result)
print(ans)

,대여일자
2021-06-01,48081
2021-06-02,56100
2021-06-03,18985
2021-06-04,56788
2021-06-05,52730
2021-06-06,47787
2021-06-07,52623
2021-06-08,55623
2021-06-09,54507
2021-06-10,36992


2021-06-04


In [5]:
# 32. 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가하고 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력
# 데이터타입 변환
df['대여일자'] = pd.to_datetime(df['대여일자'])
df['day_name'] = df['대여일자'].dt.day_name()
result = df['day_name'].value_counts().to_frame()
print(result)

           day_name
Wednesday    110607
Friday       108877
Monday       107568
Tuesday      103704
Saturday     101299
Sunday        90918
Thursday      55977


In [6]:
# 33. 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력
result = df.groupby(['day_name', '대여소번호']).size().to_frame('size').sort_values(['day_name', 'size'], ascending = False).reset_index()
ans = result.drop_duplicates('day_name', keep = 'first').reset_index(drop = True)
display(ans)

,day_name,대여소번호,size
0,Wednesday,502,282
1,Tuesday,502,267
2,Thursday,2715,137
3,Sunday,502,372
4,Saturday,502,378
5,Monday,502,242
6,Friday,502,277


In [7]:
# 34. 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라.
# 일일권의 경우 일일권 과 일일권(비회원)을 모두 포함
daily = df[df['대여구분코드'].isin(['일일권', '일일권(비회원)'])]['연령대코드'].value_counts().sort_index()
total = df['연령대코드'].value_counts().sort_index()
ratio = (daily/total).sort_values(ascending = False)
print(ratio)
print('max ratio age', ratio.index[0])

~10대    0.312525
20대     0.239913
30대     0.166702
40대     0.110585
50대     0.065414
70대~    0.048962
60대     0.042462
Name: 연령대코드, dtype: float64
max ratio age ~10대


In [8]:
# 35. 연령대별 평균 이동거리
result = df[['연령대코드', '이동거리']].groupby(['연령대코드']).mean()
print(result)

              이동거리
연령대코드             
20대    3211.890552
30대    3341.443859
40대    3514.857416
50대    3593.668100
60대    3538.145737
70대~   3085.039641
~10대   2634.426279


In [9]:
# 36. 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출
# 최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하기
tw = df[df['연령대코드'] == '20대'].reset_index(drop = True)
# 데이터의 이동거리값의 평균 이상인 데이터 추출
tw_mean = tw[tw['이동거리'] >= tw['이동거리'].mean()].reset_index(drop = True)
# 데이터타입 변환
tw_mean['탄소량'] = tw_mean['탄소량'].astype('float')
target = tw_mean.sort_values(['대여일자', '대여소번호'], ascending = False).reset_index(drop = True).iloc[:200]['탄소량']
result = round(target.sum() / len(target), 3)
print(result)

1.613


In [10]:
# 37. 6월 7일 ~10대의 “이용건수”의 중앙값
# 데이터타입 변환
df['대여일자'] = pd.to_datetime(df['대여일자'])
result = df[(df['연령대코드'] == '~10대') & (df['대여일자'] == pd.to_datetime('2021-06-07'))]['이용건수'].median()
print(result)

1.0


In [11]:
# 38. 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후
# 각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력
target = df[(df['day_name'].isin(['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Monday'])) & (df['대여시간'].isin([6, 7, 8]))]
result = target.groupby(['대여시간', '대여소번호']).size().to_frame('이용 횟수')

ans = result.sort_values(['대여시간', '이용 횟수'], ascending = False).groupby('대여시간').head(3)
display(ans)

이용 횟수
대여시간 대여소번호       
8    2701     119
     646      115
     1152      92
7    259      104
     230       77
     726       77
6    2744      45
     1125      40
     1028      36

In [12]:
# 39. 이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값 구하기
result = df[df['이동거리'] >= df['이동거리'].mean()].reset_index(drop = True)['이동거리'].std()
print(result)

5092.139707505305


In [13]:
# 40. 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값
df['sex'] = df['성별'].map(lambda x: '남' if x in ['M', 'm'] else '여')
ans = df[['sex', '이동거리']].groupby('sex').mean()
display(ans)

,이동거리
sex,
남,3209.110871
여,3468.575025


In [14]:
# 41. 전세계 행복도 지표 데이터
# 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 각년도의 행복랭킹 10위를 차지한 나라의 행복점수의 평균 구하기
# 데이터 불러오기
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv')
df.head()

,행복랭킹,나라명,점수,상대GDP,사회적지원,행복기대치,선택의 자유도,관대함,부패에 대한인식,년도
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,2019
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,2019
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,2019


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   행복랭킹      312 non-null    int64  
 1   나라명       312 non-null    object 
 2   점수        312 non-null    float64
 3   상대GDP     312 non-null    float64
 4   사회적지원     312 non-null    float64
 5   행복기대치     312 non-null    float64
 6   선택의 자유도   312 non-null    float64
 7   관대함       312 non-null    float64
 8   부패에 대한인식  311 non-null    float64
 9   년도        312 non-null    int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 24.5+ KB


In [16]:
ans = df[df['행복랭킹'] == 10]['점수'].mean()
print(ans)

7.259


In [17]:
# 42. 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 각년도의 행복랭킹 50위이내의 나라들의 각각의 행복점수 평균을 데이터프레임으로 표시
ans = df[df['행복랭킹'] <= 50][['년도', '점수']].groupby('년도').mean()
print(ans)

           점수
년도           
2018  6.64678
2019  6.67002


In [18]:
# 43. 2018년도 데이터들만 추출하여 행복점수와 부패에 대한 인식에 대한 상관계수를 구하기
ans = df[df['년도'] == 2018][['점수', '부패에 대한인식']].corr().iloc[0, 1]
print(ans)

0.40529152271510027


In [19]:
# 44. 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수를 구하여라
ans = len(df[['행복랭킹', '나라명']]) - len(df[['행복랭킹', '나라명']].drop_duplicates())
print(ans)

15


In [20]:
# 45. 2019년도 데이터들만 추출하여 각변수간 상관계수를 구하고 내림차순으로 정렬한 후 상위 5개를 데이터 프레임으로 출력하라. 컬럼명은 v1,v2,corr으로 표시
zz = df[df['년도'] == 2019].corr().unstack().to_frame().reset_index().dropna()

result = zz[zz[0] != 1].sort_values(0, ascending = False).drop_duplicates(0)
ans = result.head(5).reset_index(drop = True)
ans.columns = ['v1', 'v2', 'corr']
display(ans)

C:\Users\chunc\AppData\Local\Temp\ipykernel_65152\505997513.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  zz = df[df['년도'] == 2019].corr().unstack().to_frame().reset_index().dropna()


,v1,v2,corr
0,행복기대치,상대GDP,0.835462
1,상대GDP,점수,0.793883
2,행복기대치,점수,0.779883
3,사회적지원,점수,0.777058
4,사회적지원,상대GDP,0.754906


In [21]:
# 46. 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수
ans = df.groupby('년도').tail(5).groupby('년도')[['점수']].mean()
print(ans)

          점수
년도          
2018  3.1800
2019  3.1408


In [22]:
# 47. 2019년 데이터를 추출하고 해당데이터의 상대 GDP 평균 이상의 나라들과 평균 이하의 나라들의 행복점수 평균을 각각 구하고 그 차이값을 출력
over = df[df['상대GDP'] >= df['상대GDP'].mean()]['점수'].mean()
under = df[df['상대GDP'] <= df['상대GDP'].mean()]['점수'].mean()

result = over - under
print(result)

1.4212148925537234


In [23]:
# 48. 각년도의 부패에 대한인식을 내림차순 정렬했을때 상위 20개 국가의 부패에 대한인식의 평균
ans = df.sort_values(['년도', '부패에 대한인식'], ascending=False).groupby('년도').head(20).groupby(['년도'])['부패에 대한인식'].mean()
print(ans)

년도
2018    0.3267
2019    0.3201
Name: 부패에 대한인식, dtype: float64


In [24]:
# 49. 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자를 구하여라
result = set(df[(df['년도'] == 2018) & (df['행복랭킹'] <= 50)]['나라명']) - set(df[(df['년도'] == 2019) & (df['행복랭킹'] > 50)]['나라명'])
ans = len(result)
print(ans)

47


In [25]:
# 50. 2018년, 2019년 모두 기록이 있는 나라들 중 년도별 행복점수가 가장 증가한 나라와 그 증가 수치
count = df['나라명'].value_counts()
target = count[count >= 2].index

df2 = df.copy()
multiple = df2[df2['나라명'].isin(target)].reset_index(drop = True)
multiple.loc[multiple['년도']==2018,'점수'] = multiple[multiple['년도'] == 2018]['점수'].values * (-1)
result = multiple.groupby('나라명').sum()['점수'].sort_values().to_frame().iloc[-1]
result

점수    0.87
Name: Burundi, dtype: float64

In [26]:
# 51. 지역구 에너지 소비량 데이터
# DateTime컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 구하기
csvFile = 'C:/Users/chunc/OneDrive/바탕 화면/Python연습/power+consumption+of+tetouan+city/Tetuan City power consumption.csv'
df = pd.read_csv(csvFile)
df.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


In [27]:
# 데이터타입 변환
df['DateTime'] = pd.to_datetime(df['DateTime'])

# 월별로 몇 개의 데이터가 존재하는지 데이터프레임 형태 구현
ans = df['DateTime'].dt.month.value_counts().sort_index().to_frame()
print(ans)

    DateTime
1       4464
2       4032
3       4464
4       4320
5       4464
6       4320
7       4464
8       4464
9       4320
10      4464
11      4320
12      4320


In [28]:
# 52. 3월달의 각 시간대별 온도의 평균들 중 가장 낮은 시간대의 온도를 출력
target = df[df['DateTime'].dt.month == 3]
result = target.groupby(target['DateTime'].dt.hour)['Temperature'].mean().sort_values().values[0]
print(result)

11.506612903225806


In [29]:
# 53. 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도를 출력
target = df[df['DateTime'].dt.month == 3]
result = target.groupby(target['DateTime'].dt.hour)['Temperature'].mean().sort_values().values[-1]
print(result)

18.393602150537635


In [30]:
# 54. Zone 1 Power Consumption 컬럼의 value값의 크기가 Zone 2 Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 Humidity의 평균 구하기
ans = df[df['Zone 1 Power Consumption'] > df['Zone 2  Power Consumption']]['Humidity'].mean()
print(ans)

68.23624448055052


In [31]:
# 55. 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기
result = df.iloc[:,-3:].corr()
display(result)

,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
Zone 1 Power Consumption,1.000000,0.834519,0.750733
Zone 2 Power Consumption,0.834519,1.000000,0.570932
Zone 3 Power Consumption,0.750733,0.570932,1.000000


In [32]:
# 56. Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B,20이상 30미만의 경우 C, 그 외의 경우 D라고 할때 각 단계의 데이터 숫자 구하기
def my_data(x):
    if x < 10:
        return 'A'
    elif x < 20:
        return 'B'
    elif x < 30:
        return 'C'
    else:
        return 'D'

df['sp'] = df['Temperature'].map(my_data)
result = df['sp'].value_counts()
display(result)

B    26993
C    21105
A     2874
D     1444
Name: sp, dtype: int64

In [33]:
# 57. 6월 데이터중 12시의 Temperature의 표준편차 구하기
result = df[(df['DateTime'].dt.month == 6) & (df['DateTime'].dt.hour == 12)]['Temperature'].std()
print(result)

2.049941782795103


In [34]:
# 58. 6월 데이터중 12시의 Temperature의 분산 구하기
result = df[(df['DateTime'].dt.month == 6) & (df['DateTime'].dt.hour == 12)]['Temperature'].var()
print(result)

4.202261312849164


In [35]:
# 59. Temperature의 평균이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값 구하기
result = df[df['Temperature'] >= df['Temperature'].mean()].sort_values('Temperature')['Humidity'].values[3]
print(result)

87.9


In [36]:
# 60. Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값 구하기
result = df[df['Temperature'] >= df['Temperature'].median()].sort_values('Temperature')['Humidity'].values[3]
print(result)

80.3


In [37]:
# 61. 포켓몬 정보 데이터
# Legendary 컬럼은 전설포켓몬 유무를 나타낸다. 전설포켓몬과 그렇지 않은 포켓몬들의 HP평균의 차이를 구하여라
# 데이터 불러오기
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


In [39]:
# 전설포켓몬과 그렇지 않은 포켓몬들의 HP평균의 차이
target = df.groupby('Legendary')['HP'].mean()
result = target.values[1] - target.values[0]
print(result)

25.55614861329147


In [40]:
# 62. Type 1은 주속성 Type 2 는 부속성을 나타낸다. 가장 많은 부속성 종류는 무엇인가?
ans = df['Type 2'].value_counts().index[0]
print(ans)

Flying


In [41]:
# 63. 가장 많은 Type 1 의 종의 평균 Attack 을 평균 Defense로 나눈값
Max = df['Type 1'].value_counts().index[0]
result = df[df['Type 1'] == Max]['Attack'].mean() / df[df['Type 1'] == Max]['Defense'].mean()
print(result)

1.0165238678090576


In [42]:
# 64. 포켓몬 세대(Generation) 중 가장많은 Legendary를 보유한 세대는 몇세대인가?
ans = df[df['Legendary'] == True]['Generation'].value_counts().index[0]
print(ans)

3


In [43]:
# 65. ‘HP’, ‘Attack’, ‘Defense’, ‘Sp. Atk’, ‘Sp. Def’, ‘Speed’ 간의 상관 계수중 가장 절댓값이 큰 두 변수와 그 값을 구하여라
target = df[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].corr().unstack().reset_index().rename(columns = {0:'corr'})
result = target[target['corr'] != 1].sort_values('corr', ascending = False).iloc[0]
print(result)

level_0     Sp. Def
level_1     Defense
corr       0.510747
Name: 26, dtype: object


In [44]:
# 66. 각 Generation의 Attack으로 오름차순 정렬시 상위 3개 데이터들(18개)의 Attack의 전체 평균 구하기
result = df.sort_values(['Generation', 'Attack']).groupby('Generation').head(3)['Attack'].mean()
print(result)

19.5


In [45]:
# 67. 각 Generation의 Attack으로 내림차순 정렬시 상위 5개 데이터들(30개)의 Attack의 전체 평균 구하기
result = df.sort_values(['Generation', 'Attack'], ascending = False).groupby('Generation').head(5)['Attack'].mean()
print(result)

157.23333333333332


In [46]:
# 68. 가장 흔하게 발견되는 (Type1 , Type2) 의 쌍은 무엇인가
result = df[['Type 1', 'Type 2']].value_counts().head(1)
print(result)

Type 1  Type 2
Normal  Flying    24
dtype: int64


In [47]:
# 69. 한번씩만 존재하는 (Type1 , Type2)의 쌍의 갯수는 몇개인가
target = df[['Type 1', 'Type 2']].value_counts()
result = len(target[target == 1])
print(result)

39


In [48]:
# 70. 한번씩만 존재하는 (Type1 , Type2)의 쌍을 각 세대(Generation)은 각각 몇개씩 가지고 있는가?
target = df[['Type 1', 'Type 2']].value_counts()
target2 = target[target == 1]

lst = []
for value in target2.reset_index().values:
    t1 = value[0]
    t2 = value[1]

    sp = df[(df['Type 1'] == t1) & (df['Type 2'] == t2)]
    lst.append(sp)

result = pd.concat(lst).reset_index(drop = True)['Generation'].value_counts().sort_index()
print(result)

1     1
2     4
3     5
4    13
5     7
6     9
Name: Generation, dtype: int64
